# Test 6: Saving reference file in parquet format

## Summary:

For larger datasets, the combined reference file in JSON format can become large, e.g. for MUR 0.01 degree, the entire data set is estimated to have a 1-2 GB reference file. This notebook compares the reference file size in JSON format vs parquet, and verifies that the parquet file works as expected.

The reference files are created using kerchunk and earthaccess is used to access NASA Earthdata. 

## Results


## Install packages

To install kerchunk, used
```
!pip install git+https://github.com/fsspec/kerchunk

```
Also needed the fastparquet package to save in parquet format
```
!pip install fastparquet
```

In [1]:
import os
import fsspec
import kerchunk
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import ujson
import xarray as xr
import earthaccess
import coiled

In [18]:
earthaccess.login()
shortname = "MUR-JPL-L4-GLOB-v4.1"
granule_info = earthaccess.search_data(
    short_name=shortname,
    #temporal=("2019-01-01", "2019-12-31"),
    count=(365*5)
    )

Granules found: 8102


In [19]:
fobjs = earthaccess.open(granule_info)

Opening 1825 granules, approx size: 614.38 GB
using endpoint: https://archive.podaac.earthdata.nasa.gov/s3credentials


QUEUEING TASKS | :   0%|          | 0/1825 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1825 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1825 [00:00<?, ?it/s]

## Create all individual ref files for first five years

In [2]:
## Store reference JSONs in these directories:
dir_refs_indv = './reference_jsons_individual/'
dir_refs_comb = './reference_jsons_combined/'

In [5]:
!mkdir $dir_refs_indv
!mkdir $dir_refs_comb

In [21]:
def single_ref_earthaccess(fobj):
    """
    Inputs
    ------
    fobj: earthaccess.store.EarthAccessFile object
        Obtained from a call to earthaccess.open().
    """
    endpoint = fobj.full_name
    reference = SingleHdf5ToZarr(fobj, endpoint, inline_threshold=0).translate()
    return reference, endpoint # returns both the kerchunk reference and the path the file on podaac-ops-cumulus-protected

In [7]:
%%time

## --------------------------------------------
## Create single reference files with parallel computing using Coiled
## --------------------------------------------

# Wrap `create_single_ref` into coiled function:
single_ref_earthaccess_par = coiled.function(
    region="us-west-2", spot_policy="on-demand", 
    vm_type="t4g.large", n_workers=100
    )(single_ref_earthaccess)

# Begin computations:
fobjs_process = fobjs[:365*5]
results = single_ref_earthaccess_par.map(fobjs_process)

# Save results to JSONs as they become available:
for reference, endpoint in results:
    name_ref = dir_refs_indv + endpoint.split('/')[-1].replace('.nc', '.json')
    with open(name_ref, 'w') as outf:
        outf.write(ujson.dumps(reference))

single_ref_earthaccess_par.cluster.shutdown()

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│            ╷                                                                 │
│   Package  │ Note                                                            │
│ ╶──────────┼───────────────────────────────────────────────────────────────╴ │
│   kerchunk │ Wheel built from                                                │
│            │ /opt/coiled/env/.path-deps/kerchunk-0.2.5.post5-py3-none-any.   │
│            │ whl                                                             │
│            ╵                                                                 │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────── Not Synced with Cluster ───────────────────────────╮
│             ╷                                                    ╷           │
│   Package   │ Error                                              │ Risk      │
│ ╶───────────┼────────────────────────────────────────────────────┼─────────╴ │
│   libcxxabi │ libcxxabi~=18.1.8 has no install candidate for     │ Warning   │
│             │ Python 3.12 linux-aarch64 on conda-forge           │           │
│             ╵                                                    ╵           │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

CPU times: user 11.6 s, sys: 628 ms, total: 12.2 s
Wall time: 5min 42s


In [3]:
ref_files_indv = [dir_refs_indv+f for f in os.listdir(dir_refs_indv) if f.endswith('.json')]
ref_files_indv.sort()
ref_files_indv[:5]

['./reference_jsons_individual/20020601090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json',
 './reference_jsons_individual/20020602090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json',
 './reference_jsons_individual/20020603090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json',
 './reference_jsons_individual/20020604090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json',
 './reference_jsons_individual/20020605090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.json']

## Create combined ref files in JSON and parquet for first year, then compare

#### Create ref files

In [11]:
fs = earthaccess.get_s3fs_session(daac="PODAAC")

In [ ]:
%%time

mzz = MultiZarrToZarr(
ref_files_indv[0:365],
remote_protocol="s3",
remote_options=fs.storage_options,
concat_dims=["time"], 
#identical_dims=dims_nottime
)
ref_combined = mzz.translate()

 # Save reference info to JSON:
fname_json = dir_refs_comb + shortname + "_year" + str(i).zfill(2) + "_combined.json"
with open(fname_json, 'wb') as outf:
    outf.write(ujson.dumps(ref_combined).encode())

 # Save reference info to parquet:
fname_parq = dir_refs_comb + shortname + "_year" + str(i).zfill(2) + "_combined.parq"
kerchunk.df.refs_to_dataframe(ref_combined, fname_parq)

#### Test and compare ref files

In [4]:
refs_json_1year_combined = [
    dir_refs_comb+f for f in os.listdir(dir_refs_comb)
    if f.endswith(".json")
    ]
refs_parq_1year_combined = [
    dir_refs_comb+f for f in os.listdir(dir_refs_comb)
    if f.endswith(".parq")
    ]
print(refs_json_1year_combined)
print(refs_parq_1year_combined)

['./reference_jsons_combined/MUR-JPL-L4-GLOB-v4.1_year01_combined.json']
['./reference_jsons_combined/MUR-JPL-L4-GLOB-v4.1_year01_combined.parq']


In [31]:
%%time

## JSON:
data_from_json = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": refs_json_1year_combined[0],
            "remote_protocol": "s3",
            "remote_options": fs.storage_options
            },
        "consolidated": False
        }
)
data_from_json

CPU times: user 1.13 s, sys: 96.5 ms, total: 1.23 s
Wall time: 1.43 s


<xarray.Dataset> Size: 7TB
Dimensions:           (time: 365, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 3kB 2002-06-01T09:00:00 ... 2003-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 946GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20020601T210000Z
    time_coverage_start:        20020531T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [33]:
%%time
data_from_json['analysed_sst'].sel(time=data_from_json["time"][:100], lat=slice(-20,20), lon=slice(-20,10)).mean().compute()

CPU times: user 51.8 s, sys: 16.8 s, total: 1min 8s
Wall time: 1min 5s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(297.19800708)

In [36]:
%%time

## parquet:
data_from_parq = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": refs_parq_1year_combined[0],
            "remote_protocol": "s3",
            "remote_options": fs.storage_options
            },
        "consolidated": False
        }
)
data_from_parq

CPU times: user 19.9 ms, sys: 6.18 ms, total: 26.1 ms
Wall time: 292 ms


<xarray.Dataset> Size: 7TB
Dimensions:           (time: 365, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 3kB 2002-06-01T09:00:00 ... 2003-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 946GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20020601T210000Z
    time_coverage_start:        20020531T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [37]:
%%time
data_from_parq['analysed_sst'].sel(time=data_from_parq["time"][:100], lat=slice(-20,20), lon=slice(-20,10)).mean().compute()

CPU times: user 50.9 s, sys: 15.1 s, total: 1min 5s
Wall time: 1min 8s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(297.19800708)

In [8]:
## Compare size of JSON vs parquet
    # JSON
print(os.path.getsize(refs_json_1year_combined[0])/10**6) # in MB
    # parquet
size_parq = 0 
for path, dirs, files in os.walk(refs_parq_1year_combined[0]):
    for f in files:
        fp = os.path.join(path, f)
        size_parq += os.path.getsize(fp)
print(size_parq/10**6) # in MB

59.155636
1.848186


In [12]:
%%time

for i in range(1,6):
    mzz = MultiZarrToZarr(
    ref_files_indv[365*(i-1):365*(i)],
    remote_protocol="s3",
    remote_options=fs.storage_options,
    concat_dims=["time"], 
    identical_dims=dims_nottime
    )
    ref_combined = mzz.translate()
    
     # Save reference info to JSON:
    fname_json = dir_refs_comb + shortname + "_year" + str(i).zfill(2) + "_combined.json"
    with open(fname_json, 'wb') as outf:
        outf.write(ujson.dumps(ref_combined).encode())

     # Save reference info to parquet:
    fname_parq = dir_refs_comb + shortname + "_year" + str(i).zfill(2) + "_combined.parq"
    kerchunk.df.refs_to_dataframe(ref_combined, fname_parq, partition=True)
    #with open(fname_parq, 'wb') as outf:
    #    outf.write(ujson.dumps(ref_combined).encode())

NameError: name 'dims_nottime' is not defined

#### Create a single combined ref file for all five years
Create the ref file from the yearly ref files, rather than from all the individual (daily) ref files.

In [6]:
ref_files_comb = [dir_refs_comb+f for f in os.listdir(dir_refs_comb) if f.endswith('.json')]
ref_files_comb.sort()

In [23]:
%%time

mzz = MultiZarrToZarr(
    ref_files_comb,
    remote_protocol="s3",
    remote_options=remote_options,
    concat_dims=["time"], 
    identical_dims=dims_nottime
    )
ref_combined = mzz.translate()

 # Save reference info to JSON:
fname = dir_refs_comb + shortname + "_allyears" "_combined.json"
with open(fname, 'wb') as outf:
    outf.write(ujson.dumps(ref_combined).encode())

CPU times: user 17.6 s, sys: 1.52 s, total: 19.1 s
Wall time: 19.1 s


#### Test results
Compare computation time for the ref file from one year to the ref file for all 5 years. Compuation is the mean of a temporal/spatial subset.

In [10]:
%%time

data_allyears = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": dir_refs_comb + shortname + "_allyears" "_combined.json",
            "remote_protocol": "s3",
            "remote_options": remote_options
            },
        "consolidated": False
        }
)
data_allyears

CPU times: user 14 ms, sys: 7.31 ms, total: 21.3 ms
Wall time: 196 ms


<xarray.Dataset> Size: 33TB
Dimensions:           (time: 1825, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 15kB 2002-06-01T09:00:00 ... 2007...
Data variables:
    analysed_sst      (time, lat, lon) float64 9TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 9TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 5TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 9TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20020601T210000Z
    time_coverage_start:        20020531T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [9]:
%%time

data_oneyear = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": ref_files_comb[3],
            "remote_protocol": "s3",
            "remote_options": remote_options
            },
        "consolidated": False
        }
)
data_oneyear

CPU times: user 17 ms, sys: 3.51 ms, total: 20.5 ms
Wall time: 88.7 ms


<xarray.Dataset> Size: 7TB
Dimensions:           (time: 365, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 3kB 2004-05-31T09:00:00 ... 2005-...
Data variables:
    analysed_sst      (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    mask              (time, lat, lon) float32 946GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 2TB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20040531T210000Z
    time_coverage_start:        20040530T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [12]:
%%time
data_oneyear['analysed_sst'].sel(time=data_oneyear["time"][:100], lat=slice(-20,20), lon=slice(-20,10)).mean().compute()

CPU times: user 58.4 s, sys: 2.37 s, total: 1min
Wall time: 52.5 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(297.03368749)

In [13]:
%%time
data_allyears['analysed_sst'].sel(time=data_oneyear["time"][:100], lat=slice(-20,20), lon=slice(-20,10)).mean().compute()

CPU times: user 57.3 s, sys: 2.86 s, total: 1min
Wall time: 48 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(297.03368749)